In [ ]:
! pip install -q tensorflow_datasets

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

import matplotlib.pyplot as plt

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
for example, label in train_dataset.take(1):
    print(f'Text: {example.numpy()} \n')
    print(f'Label: {label.numpy()} \n')
    print("---")

Text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it." 

Label: 0 

---


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
    print(f"Texts : {example.numpy()[:3]} \n")
    print(f"Labels: {label.numpy()[:3]} \n")
    print("----")

Texts : [b"They made me watch this in school and it was terrible. The movie is outdated. The episodes become confusing because fact is combined with fiction to make the story more interesting.The teachers talked about it as a treat but really it was a painfully boring experience.I have read that very few people who appear in this are actors, but most of them them do what they do in the movie in real life.This accounts for cheesy acting very often. Also, very often the story becomes mildly outrageous and far-fetched. I don't like the way some of the lines were written and wish they had more meaning to them. Though, it was written to be educational, funny, suspenseful, and hip, It ended up being boring, dry, far-fetched, and old. I hope no one takes time to watch this movie because you would be just fine not seeing it."
 b'Every generation fully believes it is living in the end times. This has been true for thousands of years now. And movies like this feed on this. How did they get the g

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

vocab = np.array(encoder.get_vocabulary())

In [ ]:
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 35,  91,  70, ...,   0,   0,   0],
       [168,   1,   1, ...,   0,   0,   0],
       [ 10,  41, 208, ...,   0,   0,   0]])

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b"They made me watch this in school and it was terrible. The movie is outdated. The episodes become confusing because fact is combined with fiction to make the story more interesting.The teachers talked about it as a treat but really it was a painfully boring experience.I have read that very few people who appear in this are actors, but most of them them do what they do in the movie in real life.This accounts for cheesy acting very often. Also, very often the story becomes mildly outrageous and far-fetched. I don't like the way some of the lines were written and wish they had more meaning to them. Though, it was written to be educational, funny, suspenseful, and hip, It ended up being boring, dry, far-fetched, and old. I hope no one takes time to watch this movie because you would be just fine not seeing it."
Round-trip:  they made me watch this in school and it was terrible the movie is [UNK] the episodes become [UNK] because fact is [UNK] with [UNK] to make the story more 

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 1377s 3s/step - loss: 0.6097 - accuracy: 0.6143 - val_loss: 0.4249 - val_accuracy: 0.8234
Epoch 2/10
391/391 [==============================] - 1321s 3s/step - loss: 0.3861 - accuracy: 0.8301 - val_loss: 0.3407 - val_accuracy: 0.8521
Epoch 3/10
391/391 [==============================] - 1323s 3s/step - loss: 0.3404 - accuracy: 0.8528 - val_loss: 0.3488 - val_accuracy: 0.8224
Epoch 4/10
391/391 [==============================] - 1303s 3s/step - loss: 0.3236 - accuracy: 0.8614 - val_loss: 0.3355 - val_accuracy: 0.8427
Epoch 5/10
391/391 [==============================] - 1321s 3s/step - loss: 0.3127 - accuracy: 0.8655 - val_loss: 0.3123 - val_accuracy: 0.8740
Epoch 6/10
391/391 [==============================] - 1341s 3s/step - loss: 0.3067 - accuracy: 0.8680 - val_loss: 0.3244 - val_accuracy: 0.8630
Epoch 7/10
391/391 [==============================] - 1283s 3s/step - loss: 0.3035 - accuracy: 0.8698 - val_loss: 0.3416 - val_accuracy:

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
predictions

1/1 [==============================] - 8s 8s/step


array([[-0.00228294]], dtype=float32)

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 1298s 3s/step - loss: 0.2929 - accuracy: 0.8746 - val_loss: 0.3075 - val_accuracy: 0.8542
Epoch 2/10
391/391 [==============================] - 1329s 3s/step - loss: 0.2911 - accuracy: 0.8727 - val_loss: 0.3043 - val_accuracy: 0.8687
Epoch 3/10
391/391 [==============================] - 1255s 3s/step - loss: 0.2882 - accuracy: 0.8753 - val_loss: 0.3344 - val_accuracy: 0.8469
Epoch 4/10
391/391 [==============================] - 1267s 3s/step - loss: 0.2866 - accuracy: 0.8771 - val_loss: 0.3279 - val_accuracy: 0.8594
Epoch 5/10
385/391 [============================>.] - ETA: 18s - loss: 0.2847 - accuracy: 0.8771